In [1]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments

from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset

In [3]:
from huggingface_hub import notebook_login

notebook_login()  # same as before

In [11]:
# 0. imports
import os
from dataclasses import dataclass, field
from typing import Dict, Optional

import torch
from datasets import Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from transformers import AutoTokenizer, HfArgumentParser, TrainingArguments
from transformers import GenerationConfig
from trl import DPOTrainer

In [4]:
from datasets import load_dataset

#dataset = load_dataset("Dahoas/full-hh-rlhf",split='test')
dataset = load_dataset('json',data_files='dpo.jsonl',split = "train",use_auth_token=True)

/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


In [5]:
dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 231
})

In [6]:
dataset['chosen'][0]

"The patient's diagnosis is chronic inflammatory demyelinating polyneuropathy (CIDP)."

In [7]:
prompt = dataset['prompt'][0]
prompt


'[INST]<<SYS>>\n You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the \ncontext text provided.\nExtract the answer from the context and do not generate answer by your own.\n\n<</SYS>>\n\ngiven the following medical context and a list of diagnoses list, \\nmedical context:[\'STONE ST, CHRISTIANSBURG VA-24073-5021 Home: 540-230-5881 Guarantor: Durham, Donna Insurance: Anthem Healthkeepers Medicare Advantage PCP: Charles Judy Appointment Facility: Blue Ridge Associates in Neurology 01/25/2023 Progress Notes: Jill B Cramer, MD Current Medications Reason for Appointment Taking 1.\\nNeeds in ASAP - f/u radiculopathy - lumbar region?, muscle spasms?\\nVitarrin C Tylenol History of Present Illness Biderberry Constitutional: Levothyroxine Sodium25 MOG\\nTablet 1 tablet in the Ms Durham is a 55 yo female who presents today for restablishing with Dr. morning on an empty stomach Orally Once a day traZODone HC 100 MG Tablet 1 tablet at beditime Cramer

# SFT Model Inference

In [8]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

In [9]:

model_path = '/home/jovyan/sai/sft_biomedgpt7b'

tokenizer = AutoTokenizer.from_pretrained(model_path)


model = AutoPeftModelForCausalLM.from_pretrained(
    model_path,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.2,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
CUDA extension not installed.
CUDA extensi

In [10]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


[INST]<<SYS>>
 You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the 
context text provided.
Extract the answer from the context and do not generate answer by your own.

<</SYS>>

given the following medical context and a list of diagnoses list, \nmedical context:['STONE ST, CHRISTIANSBURG VA-24073-5021 Home: 540-230-5881 Guarantor: Durham, Donna Insurance: Anthem Healthkeepers Medicare Advantage PCP: Charles Judy Appointment Facility: Blue Ridge Associates in Neurology 01/25/2023 Progress Notes: Jill B Cramer, MD Current Medications Reason for Appointment Taking 1.\nNeeds in ASAP - f/u radiculopathy - lumbar region?, muscle spasms?\nVitarrin C Tylenol History of Present Illness Biderberry Constitutional: Levothyroxine Sodium25 MOG\nTablet 1 tablet in the Ms Durham is a 55 yo female who presents today for restablishing with Dr. morning on an empty stomach Orally Once a day traZODone HC 100 MG Tablet 1 tablet at beditime Cramer chronic infl

In [11]:
%%time
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

[INST]<<SYS>>
 You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the 
context text provided.
Extract the answer from the context and do not generate answer by your own.

<</SYS>>

given the following medical context and a list of diagnoses list, \nmedical context:['STONE ST, CHRISTIANSBURG VA-24073-5021 Home: 540-230-5881 Guarantor: Durham, Donna Insurance: Anthem Healthkeepers Medicare Advantage PCP: Charles Judy Appointment Facility: Blue Ridge Associates in Neurology 01/25/2023 Progress Notes: Jill B Cramer, MD Current Medications Reason for Appointment Taking 1.\nNeeds in ASAP - f/u radiculopathy - lumbar region?, muscle spasms?\nVitarrin C Tylenol History of Present Illness Biderberry Constitutional: Levothyroxine Sodium25 MOG\nTablet 1 tablet in the Ms Durham is a 55 yo female who presents today for restablishing with Dr. morning on an empty stomach Orally Once a day traZODone HC 100 MG Tablet 1 tablet at beditime Cramer chronic infl

# DPO Model Inference

In [8]:
model_path = '/home/jovyan/sai/biomedgpt_dpo7b'

tokenizer = AutoTokenizer.from_pretrained(model_path)

In [9]:
model = AutoModelForCausalLM.from_pretrained("/home/jovyan/sai/biomedgpt_dpo7b/final_merged_checkpoint", return_dict=True, torch_dtype=torch.float16,device_map="cuda")
#model = PeftModel.from_pretrained(model, "/home/jovyan/sai/biomedgpt_dpo7b/final_checkpoint")
#model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.2,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

In [13]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [14]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


[INST]<<SYS>>
 You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the 
context text provided.
Extract the answer from the context and do not generate answer by your own.

<</SYS>>

given the following medical context and a list of diagnoses list, \nmedical context:['STONE ST, CHRISTIANSBURG VA-24073-5021 Home: 540-230-5881 Guarantor: Durham, Donna Insurance: Anthem Healthkeepers Medicare Advantage PCP: Charles Judy Appointment Facility: Blue Ridge Associates in Neurology 01/25/2023 Progress Notes: Jill B Cramer, MD Current Medications Reason for Appointment Taking 1.\nNeeds in ASAP - f/u radiculopathy - lumbar region?, muscle spasms?\nVitarrin C Tylenol History of Present Illness Biderberry Constitutional: Levothyroxine Sodium25 MOG\nTablet 1 tablet in the Ms Durham is a 55 yo female who presents today for restablishing with Dr. morning on an empty stomach Orally Once a day traZODone HC 100 MG Tablet 1 tablet at beditime Cramer chronic infl

In [15]:
%%time
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

[INST]<<SYS>>
 You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the 
context text provided.
Extract the answer from the context and do not generate answer by your own.

<</SYS>>

given the following medical context and a list of diagnoses list, \nmedical context:['STONE ST, CHRISTIANSBURG VA-24073-5021 Home: 540-230-5881 Guarantor: Durham, Donna Insurance: Anthem Healthkeepers Medicare Advantage PCP: Charles Judy Appointment Facility: Blue Ridge Associates in Neurology 01/25/2023 Progress Notes: Jill B Cramer, MD Current Medications Reason for Appointment Taking 1.\nNeeds in ASAP - f/u radiculopathy - lumbar region?, muscle spasms?\nVitarrin C Tylenol History of Present Illness Biderberry Constitutional: Levothyroxine Sodium25 MOG\nTablet 1 tablet in the Ms Durham is a 55 yo female who presents today for restablishing with Dr. morning on an empty stomach Orally Once a day traZODone HC 100 MG Tablet 1 tablet at beditime Cramer chronic infl